In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_red = pd.read_csv("winequality-red.csv", sep=';',index_col=False)
df_red['type'] = 1
df_white = pd.read_csv("winequality-white.csv", sep=';',index_col=False)
df_white['type'] = 0
df_wine = pd.concat([df_red, df_white], ignore_index=True)
X = df_wine.drop("quality", axis=1)
y = df_wine["quality"]

In [3]:
df_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [4]:
len(df_wine['quality'].unique())

7

In [5]:
df_wine['quality'].unique()

array([5, 6, 7, 4, 8, 3, 9], dtype=int64)

In [6]:
#misturar o df para fazer df training e test aleatorios
idx = np.arange(len(df_wine))
np.random.seed(42)
np.random.shuffle(idx)
# 80% train, 20% test
split_point = int(0.8 * len(df_wine))

train_idx = idx[:split_point]
test_idx = idx[split_point:]

X_train = X.iloc[train_idx]
y_train = y.iloc[train_idx]

X_test = X.iloc[test_idx]
y_test = y.iloc[test_idx]


In [ ]:
continuous_cols = [col for col in X_train.columns if col != 'type']
binary_cols = ['type']  # ou a coluna que for binária

# Normaliza só as contínuas
scaler = (X_train[continuous_cols].mean(), X_train[continuous_cols].std())

X_train_cont = (X_train[continuous_cols] - scaler[0]) / scaler[1]
X_test_cont  = (X_test[continuous_cols]  - scaler[0]) / scaler[1]

# Junta de volta com a coluna binária (sem mexer nela)
X_train_scaled = pd.concat([X_train_cont, X_train[binary_cols]], axis=1)
X_test_scaled  = pd.concat([X_test_cont,  X_test[binary_cols]],  axis=1)

In [8]:
X_train_scaled  = np.asarray(X_train_scaled , dtype=float)
y_train = np.asarray(y_train, dtype=float)

X_test_scaled = np.asarray(X_test_scaled, dtype=float)
y_test = np.asarray(y_test, dtype=float)

In [9]:
class NN(object):
    def __init__(self):
        self.input = 12
        self.output = 7
        self.hidden_units1 = 64
        self.hidden_units2 = 32
        # initialize the matrix of weitghs
        np.random.seed(42)
        #w1 between input and hidden layer
        self.w1 = np.random.randn(self.input, self.hidden_units1) / np.sqrt(self.input)
        self.b1 = np.zeros((1, self.hidden_units1))
        self.w2 = np.random.randn(self.hidden_units1, self.hidden_units2) / np.sqrt(self.hidden_units1)
        self.b2 = np.zeros((1, self.hidden_units2))
        self.w3 = np.random.randn(self.hidden_units2, self.output) / np.sqrt(self.hidden_units2)
        self.b3 = np.zeros((1, self.output))
    def _forward_propagation(self, X):
        # Camada 1
        self.z1 = X @ self.w1 + self.b1
        self.a1 = self._Relu(self.z1)
        
        # Camada 2
        self.z2 = self.a1 @ self.w2 + self.b2
        self.a2 = self._Relu(self.z2)          
        
        # Camada de saída 
        self.z3 = self.a2 @ self.w3 + self.b3
        self.a3 = self._Softmax(self.z3)       
        
        return self.a3
    def _Relu(self, x):
        return np.maximum(x,0)

    def _Softmax(self, z):
        z = z - np.max(z, axis=1, keepdims=True)
        exp_z = np.exp(z)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def _loss(self, predict, y):
        eps = 1e-8
        y_pred = np.clip(predict, eps, 1. - eps)
        # Categorical Cross-Entropy
        loss = -np.sum(y * np.log(y_pred)) / y.shape[0]
        return loss
    def _backward_propagation(self, X, y_one_hot):
        m = X.shape[0]
        
        # Saída
        delta3 = (self.a3 - y_one_hot) / m
        self.dw3 = self.a2.T @ delta3
        self.db3 = np.sum(delta3, axis=0, keepdims=True)
        
        # Camada escondida 2
        delta2 = (delta3 @ self.w3.T) * self._derivative_Relu(self.z2)
        self.dw2 = self.a1.T @ delta2
        self.db2 = np.sum(delta2, axis=0, keepdims=True)
        
        # Camada escondida 1
        delta1 = (delta2 @ self.w2.T) * self._derivative_Relu(self.z1)
        self.dw1 = X.T @ delta1
        self.db1 = np.sum(delta1, axis=0, keepdims=True)
        
    def _derivative_Relu(self, u):
        return (u > 0).astype(float)
    def _update(self, learning_rate=0.01):
        self.w1 -= learning_rate * self.dw1
        self.w2 -= learning_rate * self.dw2
        self.w3 -= learning_rate * self.dw3
        self.b3 -= learning_rate * self.db3
        self.b2 -= learning_rate * self.db2
        self.b1 -= learning_rate * self.db1
    def train(self, X, y, interations = 1500):
        y_one_hot = self._to_one_hot(y)
    
        for i in range(interations):
            y_hat = self._forward_propagation(X)
            loss = self._loss(y_hat, y_one_hot)
            self._backward_propagation(X, y_one_hot)
            self._update()
            if i%100 == 0:
                print(f"Erro na {i} intereacao:{loss}")
    def predict(self,X):
        y_hat = self._forward_propagation(X)
        return np.argmax(y_hat, axis=1)
    def score(self, predict, y):
        cnt = np.sum(predict==y)
        return (cnt/len(y))*100
    def _to_one_hot(self, y):
        m = y.shape[0]
        y_one_hot = np.zeros((m, self.output))
        y_one_hot[np.arange(m), y.astype(int)] = 1
        return y_one_hot


In [10]:
from sklearn.preprocessing import LabelEncoder

# Crie o encoder
le = LabelEncoder()

# Fit e transform
y_train_encoded = le.fit_transform(y_train)  # [3,4,5,6,7,8,9] → [0,1,2,3,4,5,6]
y_test_encoded = le.transform(y_test)

modelo = NN()
modelo.train(X_train_scaled, y_train_encoded)
pre_y = modelo.predict(X_test_scaled)
predictions_original = le.inverse_transform(pre_y)
score = modelo.score(pre_y, y_test_encoded)
print(f'Acurácia: {score:.2f}%') 

Erro na 0 intereacao:2.128874423474061
Erro na 100 intereacao:1.354498032688303
Erro na 200 intereacao:1.254718473745836
Erro na 300 intereacao:1.2144500554426692
Erro na 400 intereacao:1.1896703357630771
Erro na 500 intereacao:1.1713769296667416
Erro na 600 intereacao:1.157167400736596
Erro na 700 intereacao:1.145706981412981
Erro na 800 intereacao:1.1361468465435196
Erro na 900 intereacao:1.1281289954365226
Erro na 1000 intereacao:1.1211672059211841
Erro na 1100 intereacao:1.114974998513293
Erro na 1200 intereacao:1.109477480806562
Erro na 1300 intereacao:1.104551664368891
Erro na 1400 intereacao:1.1001204424044952
Acurácia: 56.00%


In [ ]:

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)
y_pred = modelo.predict(X_test_scaled)

rmse = rmse(y_test_encoded, y_pred)
r2 = r2_score(y_test_encoded, y_pred)

print("RMSE:", rmse)
print("R²:", r2)

RMSE: 0.7795462190866604
R²: 0.2007060559552457
